# Clusterização de dados sobre a dengue no Brasil em 2024

Para esta clusterização foram utilizados dados disponibilizados pelo SUS referentes a casos de suspeita de dengue no ano de 2024. A escolha deste ano para a análise foi baseada na quantidade de casos registrados que, em comparação com os demais períodos, teve um pico maior. Além disso, é o último ano completo até o momento da realização deste trabalho.<br>

## Visão Geral

Os dados foram agregados por município visando a formação de possíveis clusters regionais para análise de perfil clínico e social. Todos os dados agregados são proporcionais aos casos do município para evitar que municípios com mais casos prejudiquem a análise por desbalanceamento.<br>

O dataset possui 131 colunas, porém foram extraídas somente as colunas de interesse, sendo elas:<br>
##### Localidade
 - **SG_UF**: Código do estado de residência (IBGE).
 - **ID_MN_RESI**: Código do município de residência (IBGE).
##### Informações pessoais
 - **ANO_NASC**: Ano de nascimento do paciente.
 - **CS_SEXO**: Sexo do paciente.
 - **CS_RACA**: Raça do paciente.
 - **CS_GESTANT**: Indica se a paciente é gestante.<br>
##### Sintomas
 - **VOMITO**
 - **NAUSEA**
 - **DOR_RETRO**: Dor retroorbital.
##### Comorbidades
 - **DIABETES**
 - **HEMATOLOG**
 - **HEPATOPAT**
 - **RENAL**
 - **HIPERTENSA**
 - **ACIDO_PEPT**
 - **AUTO_IMUNE**
##### Desfecho
 - **HOSPITALIZ**: Indica se o paciente foi hospitalizado.
 - **CLASSI_FIN**: Indica se o caso foi confirmado e a gravidade.
 - **EVOLUCAO**: Indica se o paciente foi curado ou veio a óbito.<br><br>

A partir destas colunas foram criadas outras métricas agregadas por município:
 - Mediana de idade
 - Porcentagem de hospitalização
 - Porcentagem de óbito por agravo
 - Porcentagem de PPI
 - Porcentagem de brancos (amarelos somam os 100% implicitamente para evitar redundância)
 - Porcentagem de homens (mulheres somam os 100% implicitamente para evitar redundância)
 - Porcentagem de gestantes
 - Porcentagem de casos com comorbidades
 - Porcentagem de casos com **nausea || vomito || dor-abdominal**
 - Porcentagem de casos de dengue com sinais de alarme
 - Porcentagem de casos de dengue grave


*São considerados somente casos confirmados: CLASSI_FIN = 10 || 11 || 12*<br>

## Bibliotecas

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler

## Limpeza e pré processamento

### Carregamento

Foram carregadas somente as colunas necessárias para a análise. O dataset também foi carregado por pedaços por conta do tamanho.

In [ ]:
desired_cols = [
  "SG_UF",      # Localidade
  "ID_MN_RESI", 

  "ANO_NASC",   # Pessoais
  "CS_SEXO",
  "CS_RACA",
  "CS_GESTANT",

  "VOMITO",     # Sintomas
  "NAUSEA",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",

  "HOSPITALIZ", # Desfecho
  "CLASSI_FIN", 
  "EVOLUCAO",
]

# Organização de diretórios
DATA_PATH = "./data/DENGBR24.csv"
CHARTS_PATH = "./charts/regional_clustering/"

os.makedirs(f"{CHARTS_PATH}", exist_ok=True)

# Carregando dataset
chunks = []

for chunk in pd.read_csv(DATA_PATH, low_memory=False, usecols=desired_cols, chunksize=500_000):
  filtered = chunk[
    (chunk["CLASSI_FIN"].isin([10, 11, 12]))  &
    (chunk["ANO_NASC"].notna())               &   
    (chunk["ANO_NASC"] > 1924)                &   
    (chunk["CS_SEXO"].isin(["M","F"]))        &
    (chunk["CS_RACA"].isin([1, 2, 3, 4, 5]))              
  ]
  chunks.append(filtered)

df = pd.concat(chunks, ignore_index=True)
df.head()

In [ ]:
df.info()

### Tratamento

In [ ]:
df_treated = df.copy()

Nesta etapa foram criadas as colunas correspondentes aos dados de interesse para a clusterização, destacados na sessão *Visão Geral*.

In [ ]:

df_treated["IDADE"] = 2024 - df_treated["ANO_NASC"]

df_treated["SINTOMAS_GO"] = (     # Sintomas variáveis / gastro_oculares
  (df_treated["NAUSEA"] == 1) | 
  (df_treated["VOMITO"] == 1) | 
  (df_treated["DOR_RETRO"] == 1)
).astype(int)

df_treated["COMORBIDADES"] = (
  (df_treated["DIABETES"] == 1)   | 
  (df_treated["HEMATOLOG"] == 1)  |
  (df_treated["HEPATOPAT"] == 1)  |
  (df_treated["RENAL"] == 1)      |
  (df_treated["HIPERTENSA"] == 1) |
  (df_treated["ACIDO_PEPT"] == 1) |
  (df_treated["AUTO_IMUNE"] == 1)
).astype(int)

df_treated["PPI"] = df_treated["CS_RACA"].isin([2, 4, 5]).astype(int)
df_treated["BRANCO"] = (df_treated["CS_RACA"] == 1).astype(int)
df_treated["MASCULINO"] = (df_treated["CS_SEXO"] == 'M').astype(int)
df_treated["OBITO_AGRAVO"] = (df_treated["EVOLUCAO"] == 2).astype(int)
df_treated["GESTANTE"] = df_treated["CS_GESTANT"].between(1, 4).astype(int)
df_treated["DENGUE_SA"] = (df_treated["CLASSI_FIN"] == 11).astype(int)
df_treated["DENGUE_GRAVE"] = (df_treated["CLASSI_FIN"] == 12).astype(int)
df_treated["HOSPITALIZACAO"] = (df_treated["HOSPITALIZ"] == 1).astype(int)

In [ ]:
clustering_cols = [
  "ID_MN_RESI", "IDADE", "SINTOMAS_GO", "COMORBIDADES", "PPI", "BRANCO", "MASCULINO",
  "OBITO_AGRAVO", "GESTANTE", "DENGUE_SA", "DENGUE_GRAVE", "HOSPITALIZACAO"
]
df_clustering = df_treated[clustering_cols]
df_clustering.head()

Na sessão abaixo os dados foram agregados por município e as colunas foram renomeadas para melhor legibilidade.

In [ ]:
df_agg = df_clustering.groupby("ID_MN_RESI").agg({
  "IDADE": "median",
  "HOSPITALIZACAO": "mean",
  "OBITO_AGRAVO": "mean",
  "PPI": "mean",
  "BRANCO": "mean",
  "MASCULINO": "mean",
  "GESTANTE": "mean",
  "COMORBIDADES": "mean",
  "SINTOMAS_GO": "mean",
  "DENGUE_SA": "mean",
  "DENGUE_GRAVE": "mean"
}).rename(columns={
  "IDADE": "mediana_idade",
  "HOSPITALIZACAO": "%_hospitalizacao",
  "OBITO_AGRAVO": "%_obito_agravo",
  "PPI": "%_ppi",
  "BRANCO": "%_brancos",
  "MASCULINO": "%_masculino",
  "GESTANTE": "%_gestante",
  "COMORBIDADES": "%_comorbidades",
  "SINTOMAS_GO": "%_sintomas_gastro_oculares",
  "DENGUE_SA": "%_dengue_sinais_alarme",
  "DENGUE_GRAVE": "%_dengue_grave"
}).reset_index()

df_agg = df_agg.rename(columns={"ID_MN_RESI": "municipio"})
df_agg.head()

## Pré clusterização

### Correlação

Matriz de correlação entre as características.

In [ ]:
df_corr = df_agg.drop(columns=["municipio"])
corr_matrix = df_corr.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlação entre Variáveis Agregadas por Município", fontsize=16)
plt.tight_layout()
plt.savefig(f"{CHARTS_PATH}corr_matrix.svg", format='svg', bbox_inches='tight')
plt.show()

Removendo colunas que não serão utilizadas na clusterização. A variável %_brancos é retirada por conta da alta correlação com a variável %_ppi.

In [ ]:
df_final = df_agg.drop(columns=["%_brancos", "municipio"])
df_final.head()

### Normalização

Normalização Z-score para evitar que algumas features, como a mediana da idade, se sobressaiam na clusterização.

In [ ]:
scaler = StandardScaler()
features_norm = scaler.fit_transform(df_final)

### Escolha do K do KMeans

#### Inércia

In [ ]:
inertia = []
K_range = range(1, 18)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=1)
    kmeans.fit(features_norm)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8,5))
plt.plot(K_range, inertia, 'bo-')
plt.xlabel('Número de clusters K')
plt.ylabel('Inércia (Soma das distâncias quadradas)')
plt.title('Método do Cotovelo')
plt.savefig(f"{CHARTS_PATH}kmeans_elbow_method.svg", format='svg', bbox_inches='tight')
plt.show()

#### Coeficiente de silhueta v1

In [ ]:
silhouette_scores = []
K_range = range(2, 18)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=1)
    labels = kmeans.fit_predict(features_norm)
    score = silhouette_score(features_norm, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(8,5))
plt.plot(K_range, silhouette_scores, 'bo-')
plt.xlabel('Número de clusters K')
plt.ylabel('Índice Silhouette')
plt.title('Índice de Silhueta')
plt.savefig(f"{CHARTS_PATH}kmeans_silhouette_score.svg", format='svg', bbox_inches='tight')
plt.show()

#### Coeficiente de silhueta v2

In [ ]:
# Trecho adaptado de 
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for n_clusters in range_n_clusters:
    
    # Create a subplot with 1 row and 2 columns
    fig, (ax1) = plt.subplots(1)
    fig.set_size_inches(8, 5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(features_norm) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=1)
    cluster_labels = clusterer.fit_predict(features_norm)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(features_norm, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(features_norm, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

## Clusterização

### KMeans (melhor resultado)

O melhor valor de K obtido de acordo com a inércia e silhueta para *random_state=1* foi **K=9**.

In [ ]:
kmeans = KMeans(n_clusters=9, random_state=1)
clusters = kmeans.fit_predict(features_norm)

#### Visualização

Utilização de PCA para tentativa de visualização dos clusters em 2 dimensões.

In [ ]:
pca_2d = PCA(n_components=2)
features_pca_2d = pca_2d.fit_transform(features_norm)

In [ ]:
plt.figure(figsize=(8,6))
scatter = plt.scatter(features_pca_2d[:, 0], features_pca_2d[:, 1], c=clusters, cmap='rocket')
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title("Clusters com KMeans (2D PCA)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.colorbar(scatter, label='Cluster')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Adicionando coluna de clusters no dataframe
df_clustered = df_agg.copy()
df_clustered["cluster"] = clusters
df_clustered.head()

#### Análises

##### Mapa de calor da porcentagem de características presentes por cluster

In [ ]:
df_stats = df_clustered.drop(columns=['municipio', 'mediana_idade'])
stats_por_cluster = df_stats.groupby('cluster').mean()
stats_por_cluster = stats_por_cluster * 100

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(stats_por_cluster.T, annot=stats_por_cluster.T.round(2).astype(str) + '%', fmt="", cmap='Reds')
plt.title("Distribuição Percentual das Variáveis por Cluster", fontsize=16)
plt.xlabel("Clusters")
plt.ylabel("Variáveis")
plt.tight_layout()
plt.savefig(f"{CHARTS_PATH}clusters_features_distribution_heatmap.svg", format='svg', bbox_inches='tight')
plt.show()

##### Gráfico de barra da porcentagem de características presentes por cluster

In [ ]:
sns.set(style="whitegrid")
n_clusters = stats_por_cluster.T.shape[1]  # número de clusters

fig, axes = plt.subplots(3, 3, figsize=(18, 12))
axes = axes.flatten()

for i, ax in enumerate(axes[:n_clusters]):
    cluster_data = stats_por_cluster.T.iloc[:, i].sort_values()
    sns.barplot(x=cluster_data.values, y=cluster_data.index, ax=ax, palette="Reds", hue=cluster_data.index)
    ax.set_title(f"Cluster {i}", fontsize=14)
    ax.set_xlabel("Prevalência (%)")
    ax.set_ylabel("Variáveis")
    ax.set_xlim(0, 100)

# Remove axes extras se o número for ímpar
for j in range(n_clusters, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle("Distribuição Percentual das Variáveis por Cluster", fontsize=18, y=1.02)
plt.savefig(f"{CHARTS_PATH}clusters_features_distribution_barplot.svg", format='svg', bbox_inches='tight')
plt.show()

Salva figuras individuais para cada subplot.

In [ ]:
os.makedirs(f"{CHARTS_PATH}individual_clusters_features_distribution", exist_ok=True)

sns.set(style="whitegrid")
n_clusters = stats_por_cluster.T.shape[1]

for i in range(n_clusters):
    cluster_data = stats_por_cluster.T.iloc[:, i].sort_values()

    # Cria figura individual
    fig, ax = plt.subplots(figsize=(6, 4))

    sns.barplot(
        x=cluster_data.values,
        y=cluster_data.index,
        ax=ax,
        hue=cluster_data.index,
        palette="Reds"
    )

    ax.set_title(f"Cluster {i}", fontsize=14)
    ax.set_xlabel("Prevalência (%)")
    ax.set_ylabel("Variáveis")
    ax.set_xlim(0, 100)

    plt.tight_layout()
    
    # Salva o gráfico individual
    filepath = f"{CHARTS_PATH}individual_clusters_features_distribution/cluster_{i}.svg"
    plt.savefig(filepath, format='svg', bbox_inches='tight')
    plt.close(fig)

##### Gráfico de barras da média da mediana de idades por cluster

In [ ]:
mean_median_age = df_clustered.groupby('cluster')['mediana_idade'].mean()
values = mean_median_age.values

# Normaliza os valores para 0-1 para aplicar no colormap
norm = (values - values.min()) / (values.max() - values.min())
colors = [cm.Reds(v) for v in norm]

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=mean_median_age.index, y=values, palette=colors)

plt.title("Média da Mediana de Idades por Cluster", fontsize=16)
plt.xlabel("Cluster")
plt.ylabel("Média da Mediana de Idade")
plt.xticks(mean_median_age.index)
plt.ylim(0, values.max() + 5)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostra os valores acima das barras
for i, v in enumerate(values):
    plt.text(i, v + 0.5, f"{v:.2f}", ha='center')

plt.tight_layout()
plt.savefig(f"{CHARTS_PATH}mean_median_age_by_cluster.svg", format='svg', bbox_inches='tight')
plt.show()

##### Mapa de calor da prevalência percentual por estado em cada cluster

Aqui foi utilizado os dois primeiros dígitos do código do município, que correspondem ao código do estado.<br>

In [ ]:
# Converte para string e extrai os 2 primeiros dígitos como código do estado
df_clustered['codigo_uf'] = df_clustered['municipio'].astype(int).astype(str).str[:2]
df_clustered['codigo_uf'] = df_clustered['codigo_uf'].astype(int)

codigo_uf_para_nome = {
    11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO',
    21: 'MA', 22: 'PI', 23: 'CE', 24: 'RN', 25: 'PB', 26: 'PE', 27: 'AL', 
    28: 'SE', 29: 'BA', 31: 'MG', 32: 'ES', 33: 'RJ', 35: 'SP', 41: 'PR', 
    42: 'SC', 43: 'RS', 50: 'MS', 51: 'MT', 52: 'GO', 53: 'DF'
}

df_clustered['estado'] = df_clustered['codigo_uf'].map(codigo_uf_para_nome)
contagem_estado = df_clustered.groupby(['cluster', 'estado']).size().unstack(fill_value=0)
percentual_estado = contagem_estado.div(contagem_estado.sum(axis=1), axis=0) * 100
plt.figure(figsize=(26, 8))
sns.heatmap(percentual_estado, annot=percentual_estado.round(2).astype(str) + '%', fmt='', cmap='rocket_r')
plt.title('Prevalência Percentual por Estado em cada Cluster')
plt.xlabel('Estado')
plt.ylabel('Cluster')
plt.tight_layout()
plt.savefig(f"{CHARTS_PATH}mun_distribution_by_state_heatmap.svg", format='svg', bbox_inches='tight')
plt.show()


##### Visualização da prevalência percentual de estados por cluster em mapa com geopandas

In [ ]:
# GeoJSON com os estados
brasil = gpd.read_file("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson")

brasil = brasil.rename(columns={"name": "estado"})
brasil["estado"] = brasil["estado"].str.upper()

In [ ]:
# Reconstruindo a tabela
df_long = percentual_estado.reset_index().melt(id_vars='cluster', var_name='estado', value_name='prevalencia')

In [ ]:
cltrs = sorted(df_long['cluster'].unique())
n = len(cltrs)

fig, axes = plt.subplots(3, 3, figsize=(20, 15))
axes = axes.flatten()

for i, cluster_id in enumerate(cltrs):
    dados_cluster = df_long[df_long['cluster'] == cluster_id]

    # Join com o mapa
    mapa = brasil.merge(dados_cluster, left_on='sigla', right_on="estado", how="left")

    # Plot
    mapa.plot(
        column="prevalencia",
        cmap="OrRd",
        linewidth=0.8,
        ax=axes[i],
        edgecolor='0.8',
        legend=False,
        missing_kwds={"color": "lightgrey", "label": "Sem dados"},
    )

    # Adicionar sigla dos estados no centroide
    for idx, row in mapa.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            x, y = row['geometry'].centroid.coords[0]
        else:  # MultiPolygon: usa o centroide geral
            x, y = row['geometry'].centroid.coords[0]
        axes[i].annotate(
            row['sigla'],
            xy=(x, y),
            ha='center',
            va='center',
            fontsize=8,
            color='black',
            weight='bold'
        )

    axes[i].set_title(f"Cluster {cluster_id}", fontsize=14)
    axes[i].axis('off')

# Remover subplots extras
for j in range(i + 1, 9):
    fig.delaxes(axes[j])

plt.suptitle("Distribuição Percentual de Municípios por Estado em cada Cluster", fontsize=18)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(f"{CHARTS_PATH}mun_distribution_by_state_map.svg", format='svg', bbox_inches='tight')
plt.show()

Salva figuras individuais para cada subplot.

In [ ]:
output_path = f"{CHARTS_PATH}clusters_combined_features_and_maps"
os.makedirs(output_path, exist_ok=True)

sns.set(style="whitegrid")
n_clusters = stats_por_cluster.T.shape[1]
cltrs = sorted(df_long['cluster'].unique())

for i in range(n_clusters):
    # Dados das features
    cluster_data = stats_por_cluster.T.iloc[:, i].sort_values()

    # Dados do mapa
    cluster_id = cltrs[i]
    dados_cluster = df_long[df_long['cluster'] == cluster_id]
    mapa = brasil.merge(dados_cluster, left_on='sigla', right_on="estado", how="left")

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 linha, 2 colunas

    # ----------- Subplot 1: Distribuição de features
    sns.barplot(
        x=cluster_data.values,
        y=cluster_data.index,
        ax=axes[0],
        hue=cluster_data.index,
        palette="Reds",
        dodge=False
    )
    axes[0].set_title("Distribuição das variáveis", fontsize=14)
    axes[0].set_xlabel("Prevalência (%)")
    axes[0].set_ylabel("Variáveis")
    axes[0].set_xlim(0, 100)

    # ----------- Subplot 2: Mapa de prevalência
    mapa.plot(
        column="prevalencia",
        cmap="OrRd",
        linewidth=0.8,
        ax=axes[1],
        edgecolor='0.8',
        legend=False,
        missing_kwds={"color": "lightgrey", "label": "Sem dados"},
    )
    for _, row in mapa.iterrows():
        if row['geometry'].is_empty:
            continue
        try:
            centroid = row['geometry'].centroid
            x, y = centroid.x, centroid.y
            axes[1].annotate(
                row['sigla'],
                xy=(x, y),
                ha='center',
                va='center',
                fontsize=8,
                color='black',
                weight='bold'
            )
        except:
            continue

    axes[1].set_title("Distribuição geográfica", fontsize=14)
    axes[1].axis('off')

    fig.suptitle(f"Cluster {cluster_id}", fontsize=16, weight='bold')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Espaço pro título geral
    combined_path = f"{output_path}/cluster_{cluster_id}.svg"
    plt.savefig(combined_path, format='svg', bbox_inches='tight')
    plt.close(fig)

### Agrupamento Espectral (resultado insatisfatório)

Utilizando *eigengap* para definir um bom valor de K, o valor mais expressivo no gráfico foi **K=2**. Com esse valor, os clusters formados não demonstraram resultados interessantes para nossa análise.

#### Escolha do K

Calculando eigengap

In [ ]:
from sklearn.neighbors import kneighbors_graph
from scipy.sparse import csgraph
from numpy import linalg as LA

# Construindo a matriz de adjacências do grafo de vizinhos mais próximos.
G = kneighbors_graph(features_norm, n_neighbors = 10, include_self = True)
A = 0.5 * (G + G.T)

# Construindo a Laplaciana Normalizada
L = csgraph.laplacian(A, normed = True).todense()

# Obtendo os autovalores da Laplaciana Normalizada
values, _ = LA.eigh(L)

# Plotando os valores dos gaps
plt.scatter([i for i in range(1, 21)], values[:20])
plt.xlabel('Índice do autovalor')
plt.ylabel('Autovalor');

#### Cluserização

In [ ]:
n_clusters = 2

spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=42)
labels_spectral = spectral.fit_predict(features_norm)

df_clustered['cluster_spectral'] = labels_spectral

In [ ]:
plt.scatter(features_pca_2d[:, 0], features_pca_2d[:, 1], c=labels_spectral, cmap='rocket')
plt.title("Clusters com Spectral Clustering")
plt.show()

In [ ]:
df_clustered.head()

In [ ]:
df_stats_spectral = df_clustered.drop(columns=['municipio', 'mediana_idade', "codigo_uf", "estado", "cluster"])
stats_por_cluster_spectral = df_stats_spectral.groupby('cluster_spectral').mean()

stats_por_cluster_spectral = stats_por_cluster_spectral * 100
plt.figure(figsize=(12, 6))
sns.heatmap(stats_por_cluster_spectral.T, annot=stats_por_cluster_spectral.T.round(2).astype(str) + '%', fmt="", cmap='rocket_r')
plt.title("Média das Variáveis por Cluster")
plt.xlabel("Clusters")
plt.ylabel("Variáveis")
plt.tight_layout()
plt.show()

### DBSCAN (resultado insatisfatório)

O valor de min_samples foi escolhido com base na dimensionalidade dos dados. Já o epsilon foi calculado com o k-distance plot. Entretanto, o algoritmo foi capaz de formar apenas um cluster. Muito se deve ao fato de os dados estarem densamente aglomerados, situação não muito favorável para este caso.

#### Escolha do eps

In [ ]:
# k = D + 1
k = 12  # igual ao min_samples 
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(features_norm)
distances, indices = neighbors_fit.kneighbors(features_norm)

distances = np.sort(distances[:, -1])  # distância até o 10º vizinho

plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.title("Gráfico k-distance para escolha do eps")
plt.xlabel("Pontos ordenados")
plt.ylabel(f"Distância ao {k}º vizinho")
plt.grid(True)
plt.show()

#### Clusterização

In [ ]:
dbscan = DBSCAN(eps=4, min_samples=k)
labels = dbscan.fit_predict(features_norm)

df_clustered['cluster_dbscan'] = labels

In [ ]:
sns.countplot(x='cluster_dbscan', data=df_clustered)
plt.title("Distribuição dos Clusters pelo DBSCAN")